## Langchain Basic Usage (LLM + ChatModel only)
Langchain的框架下面有主要分為兩種Model
- LLM
- Chat

由於Langchain並不提供任何Model，所以LLM相關的library都不在langchain-core裡面

#### Langchain最基本的用法
`Prompt -> LLM -> Response`

##### Loading environment variables

In [5]:
%load_ext dotenv
%dotenv

import os

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


##### Basic Prompts

In [6]:
# Everything starts with LLM
from langchain_core.messages import HumanMessage
msg = HumanMessage(content='茶餐廳是什麼？')

##### 使用ChatModel (e.g. TogetherAI)

In [7]:
from langchain_together import ChatTogether
model = ChatTogether(
    together_api_key=os.environ['KEY_TOGETHERAI'],
    model="Qwen/Qwen1.5-72B-Chat",
)

In [10]:
res = model.invoke([msg])

##### 利用output_parser將output變得更易讀

In [11]:
print(res)

content='茶餐廳是起源于香港的一种餐饮场所，结合了中西餐饮文化的特色。它提供多元化、价廉物美的食物選擇，通常既有中式点心、面食，也有西式的汉堡、咖啡、奶茶等。茶餐廳的菜单通常很广泛，顾客可以在一个地方享受到多种类型的餐点，适合快节奏的生活方式。茶餐厅文化在香港极其流行，并逐渐在其他地区也受到欢迎。它们通常有独特的装饰风格和快捷的服务方式，是体验香港当地风情的好去处。' response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 23, 'total_tokens': 131}, 'model_name': 'Qwen/Qwen1.5-72B-Chat', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None} id='run-f9eb908a-b528-4fd2-a74d-8c98bb558b75-0' usage_metadata={'input_tokens': 23, 'output_tokens': 108, 'total_tokens': 131}


In [12]:
from langchain_core.output_parsers import StrOutputParser

p = StrOutputParser()
# print(type(p))
print(p.invoke(res))

茶餐廳是起源于香港的一种餐饮场所，结合了中西餐饮文化的特色。它提供多元化、价廉物美的食物選擇，通常既有中式点心、面食，也有西式的汉堡、咖啡、奶茶等。茶餐廳的菜单通常很广泛，顾客可以在一个地方享受到多种类型的餐点，适合快节奏的生活方式。茶餐厅文化在香港极其流行，并逐渐在其他地区也受到欢迎。它们通常有独特的装饰风格和快捷的服务方式，是体验香港当地风情的好去处。


##### 加入SystemMessage (Instruction)

In [13]:
from langchain_core.messages import SystemMessage
system_message = """
必須以繁體中文回答
回答時只以','分隔每個項目，並以最簡短的方式回答，例如：
一,二,三,...
"""
human_message = """
請列舉茶餐廳不多過10種最熱門的食物，不肯定的不要回答
"""
msg_list = [
    SystemMessage(content=system_message),
    HumanMessage(content=human_message)
]

In [14]:
res = model.invoke(msg_list)

In [15]:
print(p.invoke(res))

菠蘿油,奶茶,西多士,炸雞翼,蛋撻,猪扒包,炒蛋多,雲吞面,牛肉丸,羅宋湯


##### Structured Data Parser
https://python.langchain.com/v0.2/docs/concepts/#output-parsers

In [16]:
from langchain.output_parsers import CommaSeparatedListOutputParser

csvp = CommaSeparatedListOutputParser()
parsed_res = csvp.invoke(res)
print(type(parsed_res))
print(parsed_res)

<class 'list'>
['菠蘿油', '奶茶', '西多士', '炸雞翼', '蛋撻', '猪扒包', '炒蛋多', '雲吞面', '牛肉丸', '羅宋湯']


##### 用PromptTemplate更方便

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "請列舉{place}不多過10種最熱門的食物，不肯定的不要回答"),
])

prompt_value = prompt_template.invoke(
    {
        'place': '日本'
    }
)

In [18]:
print(prompt_value)

messages=[SystemMessage(content="\n必須以繁體中文回答\n回答時只以','分隔每個項目，並以最簡短的方式回答，例如：\n一,二,三,...\n"), HumanMessage(content='請列舉日本不多過10種最熱門的食物，不肯定的不要回答')]


In [19]:
japan_csv = csvp.invoke(model.invoke(prompt_value))

In [20]:
print(japan_csv)

['寿司', '拉麵', '天妇罗', '烧肉', '章鱼烧', '亲子丼', '炸鸡', '刺身', '和牛', '便当']


##### 以steaming的方式回答

In [21]:
author_system_message = """你是一個日本輕小說作家，寫作風格要模仿{author}，請以用戶的名字命名主角，並以提供的關鍵字去寫一個大約400字的短篇故事。故事必須以繁體中文編寫。"""
prompt_template2 = ChatPromptTemplate.from_messages([
    ("system", author_system_message),
    ("ai", "請問你怎樣稱呼？"),
    ("human", "我的名字叫{username}"),
    ("ai", "你好{username}，請問你想以什麼作題材寫這個故事呢？"),
    ("human", "我想以{topics}為題材寫這個故事"),
])
prompt_value = prompt_template2.invoke(
    {
        "author": "川原礫",
        "username": "彌豆子",
        "topics": "鬼，捉鬼，親情，拯救世界"
    }
)
print(prompt_value)

messages=[SystemMessage(content='你是一個日本輕小說作家，寫作風格要模仿川原礫，請以用戶的名字命名主角，並以提供的關鍵字去寫一個大約400字的短篇故事。故事必須以繁體中文編寫。'), AIMessage(content='請問你怎樣稱呼？'), HumanMessage(content='我的名字叫彌豆子'), AIMessage(content='你好彌豆子，請問你想以什麼作題材寫這個故事呢？'), HumanMessage(content='我想以鬼，捉鬼，親情，拯救世界為題材寫這個故事')]


In [22]:
for res in model.stream(prompt_value):
    print(res.content, end="|")

故事|名|稱|：|《||彌|豆|子|的||詛|咒|之|影|》|

|在|一個|被|鬼|怪|肆|虐|的世界|中|，|名|為||彌|豆|子|的|少女|，|擁有|著|看|穿|邪|惡|靈|魂|的|雙|眼|。|她的|村子|被|一個|名|為|「|血|咒|鬼|王|」|的|強|大|靈|體|統|治|，|村民们|生活在|恐||懼|之下|。||彌|豆|子|的|父親|，|曾|是一位|勇|猛|的|捉|鬼|師|，|但在|與|血|咒|鬼|王|的|戰|鬥|中|失||蹤|，|留|給|她|一本|封|印|著|古老|咒|術|的|書|。

|時|光|荏|苒|，||彌|豆|子|靈|魂|深|處|的|勇|氣|被|親|情|的|牽|掛|點|燃|。|她|決|心|繼|承|父親|的|使命|，|踏上|拯救|村子|的|旅程|。|書|中的|咒|術|成為|她|唯一的|武器|，|每|一個|符|咒|都||蘊|含|著|未知|的力量|。|她|學|會|了|如何||駕||馭|靈|力|，|與|鬼|怪|對|峙|，|但|每次|戰|鬥|，|她|都不|禁|想起|與|父親|共|度|的|時|光|。

|在|一次次|的|挑|戰|中|，||彌|豆|子|逐||漸|強|大|，|她|揭|開|了|血|咒|鬼|王|的|真|面目|：|一個|被|深|層||詛|咒|的|靈|魂|，|渴望|救||贖|。|為了|拯救|這個|扭曲|的世界|，||彌|豆|子|選擇|了|寬|恕|而非|消|滅|，|她|以|親|情|的|溫|暖|和|勇|氣|的|光芒|，|試|圖|解|開|血|咒|鬼|王|身|上的|枷|鎖|。

|兩個|靈|魂|在|黑暗|中|碰撞|，|拯救|與|被|拯救|，|親|情|與|勇|氣|的|交||織|，|形成|了一|股|無法|忽|視|的力量|。|最後|，|當||彌|豆|子|的|咒|術|與|她|內|心|的|愛|相|融合|，|血|咒|鬼|王|的||詛|咒|終於|解|開|，|世界|重|歸|和平|。|而|她|，|也|找到了|失||蹤|父親|的|痕|跡|，|一個|新的|冒|險|正|等待|著|她|。

|這|就是|《||彌|豆|子|的||詛|咒|之|影|》，|一個|關於|成長|、|勇|氣|與|親|情|的故事|，|一個|少女|如何|以|愛|與|理解|拯救|世界|，|並|踏上|尋|找|父親|的|征|途|。||

#### LCEL: LangChain Expression Language
怎樣用LCEL簡化日本輕小說家的Chain呢？ 

In [ ]:
jnovel = prompt_template2 | model | p
res = jnovel.invoke(
    {
        "author": "支倉凍砂",
        "username": "亨利",
        "topics": "狼，香辛料，北方，旅行商人"
    })

In [ ]:
print(p.parse(res))

In [ ]:
jnovel2 = prompt_template2 | model | p
for res in jnovel2.stream(
    {
        "author": "支倉凍砂",
        "username": "亨利",
        "topics": "狼，香辛料，北方，旅行商人"
    }):
    print(res, end="")

##### 問題：Print的為何不再是res.content?

https://python.langchain.com/v0.2/docs/how_to/streaming/

### 偉大的LangChain: Runnable
https://python.langchain.com/v0.2/docs/concepts/#runnable-interface

#### Bonus Track: Tools Calling 😎

In [23]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

tools = [add, multiply]

In [24]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class Multiply(BaseModel):
    """Multiply two integers together."""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [35]:
# mistralai/Mixtral-8x22B-Instruct-v0.1
from langchain_openai import ChatOpenAI

better_model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["KEY_TOGETHERAI"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",)

model_with_tools = better_model.bind_tools(tools)


In [28]:
# google gemini-pro
from langchain_google_vertexai import ChatVertexAI

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "service_account.json"

better_model = ChatVertexAI(
    model="gemini-pro",
    project='vtxclass',
    location="asia-southeast1"
)
model_with_tools = better_model.bind_tools(tools)

In [37]:
query = "What is 3 * 12? And what is 11 + 49?"

model_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_5zmcexsfp4qol6jx7btscy96'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_jm696xbz41a56mxr8ugrx56m'}]

In [38]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [41]:
messages

[HumanMessage(content='What is 3 * 12? And what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6potaftdvkopvoffgh72nn3v', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_bxw5gzlgpgt01def5mzz1q1d', 'function': {'arguments': '{"a":11,"b":49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 303, 'total_tokens': 387}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e3d46cdf-ce0f-460d-ac5c-28f695ab7a07-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_6potaftdvkopvoffgh72nn3v'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_bxw5gzlgpgt01def5mzz1q1d'}], usage_metadata={'input_tokens': 303, 'output_tokens': 84, 'total_tokens': 387}),
 ToolMessage(content='36', tool_call_id='call_6potaftdvkopvoffgh72nn3v'),
 Tool

In [42]:
model_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uhod2gi1ukjmyzzp5xw15zjz', 'function': {'arguments': '{"a":10,"b":50}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 431, 'total_tokens': 475}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ea7980af-644c-4051-a2f6-9880401b756f-0', tool_calls=[{'name': 'add', 'args': {'a': 10, 'b': 50}, 'id': 'call_uhod2gi1ukjmyzzp5xw15zjz'}], usage_metadata={'input_tokens': 431, 'output_tokens': 44, 'total_tokens': 475})